<a href="https://colab.research.google.com/github/teploe-odealko/meteoinfo_parsing/blob/main/covid_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
filename = files.upload()
filename = next(iter(filename))

Saving МИР.xlsx to МИР (1).xlsx


#Подготовка данных с репозитория университета Хопкинса

In [2]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

In [3]:
from openpyxl.worksheet.worksheet import Worksheet
import pandas as pd
import glob
from datetime import datetime, timedelta
from copy import copy
import openpyxl
from openpyxl.formula.translate import Translator
from openpyxl.utils.cell import get_column_letter
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows

WEEKLY_REPORT_DAY = 5
countries_map = {"Taiwan*":"Тайвань", "Timor-Leste":"Восточный Тимор", "US":"США", "West Bank and Gaza":"Западный берег", "Syria": "Сирия", "Slovakia": "Словакия", "Sao Tome and Principe": "Сан-Томе и Принсипи", "Russia": "Россия", "Micronesia": "Федеративные Штаты Микронезии", "Korea, South": "Южная Корея", "Iran": "Иран", "Eswatini": "Эсватини", "Cote d'Ivoire": "Кот-д'Ивуар", "Congo (Kinshasa)": "ДР Конго", "Congo (Brazzaville)":"Браззавиль", "Cabo Verde":"Кабо-Верде", "Burma": "Мьянма", "Afghanistan":  "Афганистан", "Albania":  "Албания", "Algeria":  "Алжир", "American Samoa":  "Американское Самоа", "Andorra":  "Андорра", "Angola":  "Ангола", "Anguilla":  "Ангилья", "Antarctica":  "Антарктика", "Antigua and Barbuda":  "Антигуа и Барбуда", "Argentina":  "Аргентина", "Armenia":  "Армения", "Aruba":  "Аруба", "Australia":  "Австралия", "Austria":  "Австрия", "Azerbaijan":  "Азербайджан", "Bahamas":  "Багамские Острова", "Bahrain":  "Бахрейн", "Bangladesh":  "Бангладеш", "Barbados":  "Барбадос", "Belarus":  "Белоруссия", "Belgium":  "Бельгия", "Belize":  "Белиз", "Benin":  "Бенин", "Bermuda":  "Бермудские Острова", "Bhutan":  "Бутан", "Bolivia":  "Боливия", "Bosnia and Herzegovina":  "Босния и Герцеговина", "Botswana":  "Ботсвана", "Brazil":  "Бразилия", "Brunei":  "Бруней", "Bulgaria":  "Болгария", "Burkina Faso":  "Буркина-Фасо", "Burundi":  "Бурунди", "Cambodia":  "Камбоджа", "Cameroon":  "Камерун", "Canada":  "Канада", "Cape Verde":  "Кабо-Верде", "Cayman Islands":  "Острова Кайман", "Central African Republic":  "Центральноафриканская Республика", "Chad":  "Чад", "Chile":  "Чили", "China":  "Китай", "Colombia":  "Колумбия", "Comoros":  "Коморские острова", "Democratic Republic of the Congo (Kinshasa)":  "Демократическая Республика Конго", "Congo, Republic of (Brazzaville)":  "Республика Конго", "Cook Islands":  "Острова Кука", "Costa Rica":  "Коста-Рика", "Ivory Coast (Cote d'Ivoire)":  "Кот-д'Ивуар", "Croatia":  "Хорватия", "Cuba":  "Куба", "Cyprus":  "Кипр (остров)", "Czechia":  "Чехия", "Denmark":  "Дания", "Djibouti":  "Джибути", "Dominica":  "Доминика", "Dominican Republic":  "Доминиканская Республика", "East Timor (Timor-Leste)":  "Восточный Тимор", "Ecuador":  "Эквадор", "Egypt":  "Египет", "El Salvador":  "Сальвадор", "Equatorial Guinea":  "Экваториальная Гвинея", "Eritrea":  "Эритрея", "Estonia":  "Эстония", "Ethiopia":  "Эфиопия", "Falkland Islands":  "Фолклендские острова", "Faroe Islands":  "Фарерские острова", "Fiji":  "Фиджи", "Finland":  "Финляндия", "France":  "Франция", "French Guiana":  "Гвиана", "French Polynesia":  "Французская Полинезия", "Gabon":  "Габон", "Gambia":  "Гамбия", "Georgia":  "Грузия", "Germany":  "Германия", "Ghana":  "Гана", "Gibraltar":  "Гибралтар", "Greece":  "Греция", "Greenland":  "Гренландия", "Grenada":  "Гренада", "Guadeloupe":  "Гваделупа", "Guam":  "Гуам", "Guatemala":  "Гватемала", "Guinea":  "Гвинея", "Guinea-Bissau":  "Гвинея-Бисау", "Guyana":  "Гайана", "Haiti":  "Гаити", "Honduras":  "Гондурас", "Hong Kong":  "Гонконг", "Hungary":  "Венгрия", "Iceland":  "Исландия", "India":  "Индия", "Indonesia":  "Индонезия", "Iran (Islamic Republic of)":  "Иран (Персия)", "Iraq":  "Ирак", "Ireland":  "Ирландия", "Israel":  "Израиль", "Italy":  "Италия", "Jamaica":  "Ямайка", "Japan":  "Япония", "Jordan":  "Иордания", "Kazakhstan":  "Казахстан", "Kenya":  "Кения", "Kiribati":  "Кирибати", "Korea, Democratic People's Rep. (North Korea)":  "Северная Корея (КНДР)", "Korea, Republic of (South Korea)":  "Ю́жная Коре́я (Республика Корея)", "Kosovo":  "Косово (Республика Косово)", "Kuwait":  "Кувейт", "Kyrgyzstan":  "Киргизия", "Laos":  "Лаос", "Latvia":  "Латвия", "Lebanon":  "Ливан", "Lesotho":  "Лесото", "Liberia":  "Либерия", "Libya":  "Ливия", "Liechtenstein":  "Лихтенштейн", "Lithuania":  "Литва", "Luxembourg":  "Люксембург", "Macau":  "Макао (Аомынь)", "North Macedonia":  "Северная Македония", "Madagascar":  "Мадагаскар", "Malawi":  "Малави", "Malaysia":  "Малайзия", "Maldives":  "Мальдивы", "Mali":  "Мали", "Malta":  "Мальта", "Marshall Islands":  "Маршалловы Острова", "Martinique":  "Мартиника", "Mauritania":  "Мавритания", "Mauritius":  "Маврикий", "Mayotte":  "Майотта", "Mexico":  "Мексика", "Микронезия, Federated States of":  "Федеративные Штаты Микронезии", "Moldova":  "Молдавия", "Monaco":  "Монако, Княжество Монако", "Mongolia":  "Монголия", "Montenegro":  "Черногория", "Montserrat":  "Монтсеррат", "Morocco":  "Марокко", "Mozambique":  "Мозамбик", "Myanmar, Burma":  "Мьянма, (Би́рма)", "Namibia":  "Намибия", "Nauru":  "Науру", "Nepal":  "Непал", "Netherlands":  "Нидерланды, Голландия", "New Caledonia":  "Новая Каледония", "New Zealand":  "Новая Зеландия", "Nicaragua":  "Никарагуа", "Niger":  "Нигер", "Nigeria":  "Нигерия", "Northern Mariana Islands":  "Северные Марианские", "":  "острова", "Norway":  "Норвегия", "Oman":  "Оман", "Pakistan":  "Пакистан", "Palau":  "Палау", "State of Palestine":  "Государство Палестина", "Panama":  "Панама", "Papua New Guinea":  "Папуа — Новая Гвинея", "Paraguay":  "Парагвай", "Peru":  "Перу", "Philippines":  "Филиппины", "Poland":  "Польша", "Portugal":  "Португалия", "Puerto Rico":  "Пуэрто-Рико", "Qatar":  "Катар", "Réunion":  "Реюньон", "Romania":  "Румыния", "Russian Federation":  "Россия, Росси́йская Федера́ция", "Rwanda":  "Руанда", "Saint Kitts and Nevis":  "Сент-Китс и Невис", "Saint Lucia":  "Сент-Люсия", "Saint Vincent and the Grenadines":  "Сент-Винсент и Гренадины", "Samoa":  "Самоа", "San Marino":  "Сан-Марино", "Sao Tome and Príncipe":  "Сан-Томе и Принсипи", "Saudi Arabia":  "Саудовская Аравия", "Senegal":  "Сенегал", "Serbia":  "Сербия", "Seychelles":  "Сейшельские Острова", "Sierra Leone":  "Сьерра-Леоне", "Singapore":  "Сингапур", "Slovakia (Slovak Republic)":  "Словакия", "Slovenia":  "Словения", "Solomon Islands":  "Соломоновы Острова", "Somalia":  "Сомали", "South Africa":  "Южная Африка", "South Sudan":  "Южный Судан", "Spain":  "Испания", "Sri Lanka":  "Шри-Ланка (ранее Цейлон)", "Sudan":  "Судан", "Suriname":  "Суринам", "Swaziland (Eswatini)":  "Сва́зиле́нд (Эсватини)", "Sweden":  "Швеция", "Switzerland":  "Швейцария", "Syria, Syrian Arab Republic":  "Сирия", "Taiwan (Republic of China)":  "Китайская Республика (Тайвань)", "Tajikistan":  "Таджикистан", "Tanzania":  "Танзания", "Thailand":  "Таиланд (ранее Сиам)", "Tibet":  "Тибет", "Timor-Leste (East Timor)":  "Восточный Тимор", "Togo":  "Того", "Tokelau":  "Токелау", "Tonga":  "Тонга", "Trinidad and Tobago":  "Тринидад и Тобаго", "Tunisia":  "Тунис", "Turkey":  "Турция", "Turkmenistan":  "Туркмения", "Tuvalu":  "Тувалу", "Uganda":  "Уганда", "Ukraine":  "Украина", "United Arab Emirates":  "Объединённые Арабские Эмираты (ОАЭ)", "United Kingdom":  "Великобритания (Соединённое Короле́вство)", "United States":  "Соединённые Штаты Америки (Аме́рика, США)", "Uruguay":  "Уругвай", "Uzbekistan":  "Узбекистан", "Vanuatu":  "Вануату", "Vatican City State (Holy See)":  "Ватикан (Святой Престол)", "Venezuela":  "Венесуэла", "Vietnam":  "Вьетнам", "Virgin Islands (British)":  "Виргинские Острова (Великобритания)", "Virgin Islands (U.S.)":  "Виргинские Острова (США)", "Wallis and Futuna Islands":  "Уоллис и Футуна", "Western Sahara":  "Западная Сахара", "Yemen":  "Йемен", "Zambia":  "Замбия", "Zimbabwe":  "Зимбабве"}

In [4]:
path = r'/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename_csv in all_files:
    df = pd.read_csv(filename_csv, index_col=None, header=0)
    df['Last_Update'] = filename_csv.split('.')[0].split('/')[-1]
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame['Last_Update'] = pd.to_datetime(frame['Last_Update'], format='%m-%d-%Y')

In [5]:
def translate_country(eng):
    try:
        return countries_map[eng]
    except KeyError:
        return eng


frame['Country_Region'] = frame['Country_Region'].apply(translate_country)

In [6]:
df = frame[['Confirmed', 'Recovered', 'Deaths', 'Country_Region', 'Last_Update', 'Combined_Key']]
sorted_df = df.groupby(['Last_Update', 'Country_Region']).sum().reset_index().sort_values(by=['Country_Region', 'Last_Update'])


In [7]:
li = []
for country in sorted_df['Country_Region'].unique():
    country_df = sorted_df[sorted_df['Country_Region'] == country].copy()
    country_df_diff = country_df[['Confirmed', 'Recovered', 'Deaths']].diff()
    country_df['Confirmed_diff'] = country_df_diff['Confirmed']
    country_df['Recovered_diff'] = country_df_diff['Recovered']
    country_df['Deaths_diff'] = country_df_diff['Deaths']
    li.append(country_df)


df_diff = pd.concat(li, axis=0, ignore_index=True)
df_diff = df_diff.fillna(0)
df_diff = df_diff[['Country_Region', 'Last_Update', 'Confirmed', 'Recovered', 'Deaths', 'Confirmed_diff', 'Recovered_diff', 'Deaths_diff']]
# df_diff.to_csv('counties.csv', index=False, header=False)

#Заполнение таблиц

In [8]:
def base_countries_sheet(ws: Worksheet):
    for i, row in enumerate(dataframe_to_rows(df_diff, index=False, header=False)):
        ws.append(row)
        r = i + 2
        ws.cell(row=r, column=9).value = fr'=IF(YEAR(B{r})=2020,(IF(J{r}=1,WEEKNUM(B{r},2),IF(J{r}=2,WEEKNUM(B{r},2),IF(J{r}=3,WEEKNUM(B{r},2),IF(J{r}=4,WEEKNUM(B{r},2),IF(J{r}=5,WEEKNUM(B{r},2),IF(J{r}=6,WEEKNUM(B{r},2),IF(J{r}=7,WEEKNUM(B{r},2)+1)))))))),IF(YEAR(B{r})=2021,(IF(J{r}=1,WEEKNUM(B{r},2),IF(J{r}=2,WEEKNUM(B{r},2),IF(J{r}=3,WEEKNUM(B{r},2),IF(J{r}=4,WEEKNUM(B{r},2),IF(J{r}=5,WEEKNUM(B{r},2),IF(J{r}=6,WEEKNUM(B{r},2),IF(J{r}=7,WEEKNUM(B{r},2)+1))))))))+52))'
        ws.cell(row=r, column=10).value = fr'=WEEKDAY(B{r},2)'
        ws.cell(row=r, column=2).number_format = 'DD.MM.YYYY'

In [9]:
def continue_formula_n_down(ws: Worksheet, columns: list, row: int, n: int):
    for column in columns:

        formula = ws.cell(row=(row - n), column=column).value
        # print(row-n, column, formula)
        last_cell_id = get_column_letter(column) + str(row - n)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_formula_down(ws: Worksheet, columns: list, row: int):
    continue_formula_n_down(ws, columns, row, 1)

In [10]:
def  continue_date_down(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]][3:].index(None) + 4
        except ValueError:
            column = ws.max_column + 1
        last_table_date = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = (last_table_date) + timedelta(days=1)
        ws.cell(row=row, column=column).number_format = 'DD.MM.YY'

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

In [11]:
def continue_number_down(ws: Worksheet, columns: list):
    for column in columns:

        row = ws.max_row + 1
        last_num = ws.cell(row=row - 1, column=column).value
        ws.cell(row=row, column=column).value = last_num + 1

        last_cell_id = get_column_letter(column) + str(row-1)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

In [12]:
def base_world_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    ws.cell(row=new_row, column=1).value = datetime.now()
    ws.cell(row=new_row, column=1).number_format = 'DD.MM.YY'
    continue_formula_down(ws, list(range(2, 6)) + [7, 8, 10, 11] + list(range(13, 21)), new_row)

    today = (datetime.today() - timedelta(days=1))
    daily_df = df_diff.groupby('Last_Update').sum().loc[today.strftime('%d.%m.%y')]
    ws.cell(row=new_row, column=6).value = daily_df['Confirmed']
    ws.cell(row=new_row, column=9).value = daily_df['Recovered']
    ws.cell(row=new_row, column=12).value = daily_df['Deaths']

In [13]:
def weeks_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    # print(new_row)
    continue_number_down(ws, [1])
    continue_formula_down(ws, list(range(2, 15)), new_row)


In [14]:
def countrie_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    # print(new_row)
    continue_number_down(ws, [1])
    continue_formula_down(ws, list(range(2, 32)), new_row)

In [15]:
def main(filename: str):
    wb = openpyxl.load_workbook(filename)

    base_countries_sheet(wb['База страны'])
    base_world_sheet(wb['База мир'])

    if (datetime.today().weekday() + 1 == WEEKLY_REPORT_DAY):
        weeks_sheet(wb['Недели'])
        for country in wb.sheetnames[8:-1]:
            countrie_sheet(wb[country])

    wb.save('out.xlsx')

In [17]:
main(filename)

70
80
80
80
80
80
80
80
80
80
80
80
80
80
80


#Результат

In [18]:
files.download('out.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>